In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from datetime import datetime, timedelta
from PIL import Image
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling

In [2]:
tiff_file_input = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/256_20_chips_GSA-DE_BRB-2019_cropMask_lines_touch_true_lines_touch_true_linecrop_prediction_extent.tif'
outPath = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/2019_cropMask_lines_touch_true_lines_touch_true_linecrop_prediction_extent'
output_kml_path = f'{outPath}.kml'

In [ ]:
color_ramp = [
    (0, "#000004"),
    (0.0196078, "#02020b"),
    (0.0392157, "#050416"),
    (0.0588235, "#090720"),
    (0.0784314, "#0e0b2b"),
    (0.09803920000000001, "#140e36"),
    (0.117647, "#1a1036"),
    (0.13725499999999999, "#21114e"),
    (0.156863, "#29115a"),
    (0.17647099999999999, "#311165"),
    (0.196078, "#390f6e"),
    (0.21568599999999999, "#420f75"),
    (0.235294, "#4a1079"),
    (0.25490200000000002, "#52137c"),
    (0.27450999999999998, "#5e197e"),
    (0.29411799999999999, "#661c80"),
    (0.31372499999999998, "#6a1d81"),
    (0.33333299999999999, "#6e1f81"),
    (0.352941, "#792281"),
    (0.37254900000000002, "#812381"),
    (0.39215699999999998, "#892580"),
    (0.41176499999999999, "#912780"),
    (0.43137300000000001, "#992980"),
    (0.45097999999999999, "#a22f7e"),
    (0.47058800000000001, "#aa317d"),
    (0.49019600000000002, "#b2377c"),
    (0.50980400000000003, "#ba3d79"),
    (0.52941199999999999, "#c24376"),
    (0.54901999999999995, "#ca4b72"),
    (0.56862699999999999, "#d3536c"),
    (0.58823499999999995, "#db5c67"),
    (0.60784300000000002, "#e26761"),
    (0.62745099999999998, "#e9735a"),
    (0.64705900000000005, "#ef7e55"),
    (0.66666700000000001, "#f38b51"),
    (0.68627499999999997, "#f7984e"),
    (0.70588200000000001, "#fba54c"),
    (0.72548999999999997, "#fdb04c"),
    (0.74509800000000004, "#feb94f"),
    (0.764706, "#fdc259"),
    (0.78431399999999996, "#fdcb65"),
    (0.80392200000000003, "#fdd371"),
    (0.82352899999999996, "#fddb7e"),
    (0.84313700000000003, "#fee28b"),
    (0.86274499999999998, "#ffe998"),
    (0.88235300000000005, "#ffefa5"),
    (0.90196100000000001, "#fff5b3"),
    (0.92156899999999997, "#fffbc0"),
    (0.94117600000000001, "#fffecd"),
    (0.96078399999999997, "#fffdda"),
    (0.98039200000000004, "#fffbe6"),
    (1, "#fff9ef")
]


# Quick funktion to convert to split hex into RGBA colors, as needed later
def hex_to_rgba(hex_color, alpha=255):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (alpha,)

def map_value_to_color(value):
    for upper_bound, hex_color in color_ramp:
        if value <= upper_bound:
            return hex_to_rgba(hex_color)
    return (0, 0, 0, 0)

ds = gdal.Open(tiff_file_input, 0)
bands = ds.RasterCount
png_files = []
for i in range(bands):
    # band = ds.GetRasterBand(i + 1).ReadAsArray()

    # # Create an empty RGBA array
    # rgba_image = np.zeros((band.shape[0], band.shape[1], 4), dtype=np.uint8)

    # # Flatten band1 and map each pixel's value to color
    # flat_band = band.flatten()

    # print('create array')
    # # Create an array to hold RGBA values
    # colors = np.array([map_value_to_color(val) for val in flat_band], dtype=np.uint8)
    # print('start reshape')
    # # Reshape colors to image shape
    # rgba_image = colors.reshape((band.shape[0], band.shape[1], 4))
    # print('start conversion')
    # # Convert numpy RGBA array to PIL Image and save
    # im = Image.fromarray(rgba_image, mode="RGBA")
    # print('write away')
    # im.save(f'{outPath}_band_{i+1}.png', "PNG")
    png_files.append(f'{outPath}_band_{i+1}.png')


In [ ]:
## get bounding boxes
bboxes = []
with rasterio.open(tiff_file_input) as dataset:
    bounds = dataset.bounds
    bboxes.append(bounds)

from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom.minidom import parseString


##### insert time dummy
times = []
# Monday
first_day_of_year= datetime.fromisocalendar(2019, 1, 1)
# Sunday
last_day_of_year = first_day_of_year + timedelta(days=365)

# 1 TIF = 1 calendar week in my case
start_time = first_day_of_year.strftime("%Y-%m-%dT00:00:00Z")
end_time = last_day_of_year.strftime("%Y-%m-%dT23:59:59Z")

times.append((start_time, end_time))

In [ ]:
kml_files = []

kml_start = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://earth.google.com/kml/2.0">
<Document>"""

kml_end = """</Document>
</kml>"""

filled_kml = ""

for png_filename, (west, south, east, north), (begin, end) in zip([png_files[0]], bboxes, times):    
    kml_template = """
  <Folder>
      <name>Raster visibility</name>
      <TimeSpan><begin>{begin}</begin><end>{end}</end></TimeSpan>
      <Folder>
        <name>Raster</name>
        <GroundOverlay>
            <name>Raster data</name>
            <LatLonBox>
              <north>{north}</north>
              <south>{south}</south>
              <west>{west}</west>
              <east>{east}</east>
            </LatLonBox>
            <Icon>
              <href>{href}</href>
            </Icon>
        </GroundOverlay>
      </Folder>
    </Folder>"""
    filled_kml += kml_template.format(
        begin=begin,
        end=end,
        north=north,
        south=south,
        west=west,
        east=east,
        href=png_filename
      )

filled_kml = f"{kml_start}{filled_kml}{kml_end}"

with open(output_kml_path, 'w') as f:
    f.write(filled_kml)
print(f"KML file {output_kml_path} generated successfully")


In [ ]:
# generate the kmz file
import zipfile

base_name = os.path.splitext(os.path.basename(output_kml_path))[0]
kmz_path = os.path.join(os.path.dirname(output_kml_path), base_name + '.kmz')

with zipfile.ZipFile(kmz_path, 'w', zipfile.ZIP_DEFLATED) as kmz:
    kmz.write(output_kml_path, os.path.basename(output_kml_path))
    for png_path in [png_files[0]]:    
        kmz.write(png_path, os.path.basename(png_path))

print(f"KMZ created at: {kmz_path}")



In [3]:
import subprocess
from osgeo import gdal

# input_file = tiff_file_input


# src_ds = gdal.Open(input_file)
# band1 = src_ds.GetRasterBand(1)
# cols = src_ds.RasterXSize
# rows = src_ds.RasterYSize
# geotransform = src_ds.GetGeoTransform()
# projection = src_ds.GetProjection()
# datatype = band1.DataType

# # Create output file with 1 band
# driver = gdal.GetDriverByName('GTiff')
# dst_ds = driver.Create('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/fields.tif', cols, rows, 1, datatype)
# dst_ds.SetGeoTransform(geotransform)
# dst_ds.SetProjection(projection)

# # Read and write data
# data = band1.ReadAsArray()
# dst_ds.GetRasterBand(1).WriteArray(data)

# # Copy nodata value if present
# nodata = band1.GetNoDataValue()
# if nodata is not None:
#     dst_ds.GetRasterBand(1).SetNoDataValue(nodata)

# # Flush and close
# dst_ds.FlushCache()
# dst_ds = None
# src_ds = None

input_file = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/fields.tif'
warped_file = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/warped.tif'
input_file_scaled = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/input_scaled.tif'
rgb_file = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/rgb_rendered.tif'
output_cog = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/output_cog.tif'



ds = gdal.Open(input_file, gdal.GA_Update)
band = ds.GetRasterBand(1)
stats = band.GetStatistics(True, True)
min_val, max_val = stats[0], stats[1]


# Step 1: Reproject to WGS84 and convert to Byte
subprocess.run([
    "gdal_translate",
    "-ot", "Byte",
    "-scale", str(min_val), str(max_val), "0", "255",
    input_file,
    input_file_scaled  # Overwrite or use a new filename if needed
], check=True)

subprocess.run([
    "gdalwarp",
    "-t_srs", "EPSG:4326",
    "-ot", "Byte",
    input_file_scaled,
    warped_file
], check=True)


# Step 2: Apply color table manually using GDAL Python API
ds = gdal.Open(warped_file, gdal.GA_Update)
band = ds.GetRasterBand(1)
# Create color table
ct = gdal.ColorTable()

# Original sparse QGIS colormap (value in range [0–1], R, G, B)
qgis_colormap = [
    (0.0, 0, 0, 4),
    (0.0196, 2, 2, 11),
    (0.0392, 5, 4, 22),
    (0.0588, 9, 7, 32),
    (0.0784, 14, 11, 43),
    (0.0980, 20, 14, 54),
    (0.1176, 26, 16, 66),
    (0.1373, 33, 17, 78),
    (0.1569, 41, 17, 90),
    (0.1765, 49, 17, 101),
    (0.1961, 57, 15, 110),
    (0.2157, 66, 15, 117),
    (0.2353, 74, 16, 121),
    (0.2549, 82, 19, 124),
    (0.2745, 90, 22, 126),
    (0.2941, 98, 25, 128),
    (0.3137, 106, 28, 129),
    (0.3333, 114, 31, 129),
    (0.3529, 121, 34, 130),
    (0.3725, 129, 37, 129),
    (0.3922, 137, 40, 129),
    (0.4118, 145, 43, 129),
    (0.4314, 153, 45, 128),
    (0.4510, 161, 48, 126),
    (0.4706, 170, 51, 125),
    (0.4902, 178, 53, 123),
    (0.5098, 186, 56, 120),
    (0.5294, 194, 59, 117),
    (0.5490, 202, 62, 114),
    (0.5686, 210, 66, 111),
    (0.5882, 217, 70, 107),
    (0.6078, 224, 76, 103),
    (0.6275, 231, 82, 99),
    (0.6471, 236, 88, 96),
    (0.6667, 241, 96, 93),
    (0.6863, 244, 105, 92),
    (0.7059, 247, 114, 92),
    (0.7255, 249, 123, 93),
    (0.7451, 251, 133, 96),
    (0.7647, 252, 142, 100),
    (0.7843, 253, 152, 105),
    (0.8039, 254, 161, 110),
    (0.8235, 254, 170, 116),
    (0.8431, 254, 180, 123),
    (0.8627, 254, 189, 130),
    (0.8824, 254, 198, 138),
    (0.9020, 254, 207, 146),
    (0.9216, 254, 216, 154),
    (0.9412, 253, 226, 163),
    (0.9608, 253, 235, 172),
    (0.9804, 252, 244, 182),
    (1.0,   252, 253, 191)
]

# Interpolate to create full 256-entry color table
values = [int(round(v[0] * 255)) for v in qgis_colormap]
reds   = [v[1] for v in qgis_colormap]
greens = [v[2] for v in qgis_colormap]
blues  = [v[3] for v in qgis_colormap]

# Full range
all_vals = np.arange(256)
r_interp = np.interp(all_vals, values, reds).astype(int)
g_interp = np.interp(all_vals, values, greens).astype(int)
b_interp = np.interp(all_vals, values, blues).astype(int)

# Apply interpolated color table
ct = gdal.ColorTable()
for i in range(256):
    ct.SetColorEntry(i, (r_interp[i], g_interp[i], b_interp[i], 255))

# Apply to band
band.SetRasterColorTable(ct)
band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)
band.SetNoDataValue(0)
# Save and close
ds = None


# Step 3: Expand palette to RGB using subprocess
subprocess.run([
    "gdal_translate",
    "-expand", "rgba",
    warped_file,
    rgb_file
], check=True)

# Step 4: Convert to COG
subprocess.run([
    "gdal_translate",
    "-of", "COG",
    "-co", "COMPRESS=DEFLATE",
    "-co", "RESAMPLING=NEAREST",
    "-co", "OVERVIEWS=IGNORE_EXISTING",
    rgb_file,
    output_cog
], check=True)

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Input file size is 29972, 26904
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 37306P x 20583L.
Processing /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/input_scaled.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 37306, 20583
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 37306, 20583
0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdal_translate', '-of', 'COG', '-co', 'COMPRESS=DEFLATE', '-co', 'RESAMPLING=NEAREST', '-co', 'OVERVIEWS=IGNORE_EXISTING', '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/rgb_rendered.tif', '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/output_cog.tif'], returncode=0)

In [7]:
def print_stats(path, label):
    ds = gdal.Open(path)
    band = ds.GetRasterBand(1)
    stats = band.GetStatistics(True, True)  # [min, max, mean, std]
    print(f"{label}: min={stats[0]:.3f}, max={stats[1]:.3f}, mean={stats[2]:.3f}")
    ds = None

print_stats(input_file, "Original")
print_stats(warped_file, "Warped")

Original: min=0.000, max=1.000, mean=0.136
Warped: min=0.000, max=1.000, mean=0.131


In [9]:
max_val


1.0